In [45]:
import numpy

class matrix_factorization():
    
    def __init__(self,R,a,b,it,k):
        self.R=R
        self.user_n, self.item_n=R.shape
        self.alpha=a
        self.beta=b
        self.iterations=it
        self.K=k
        
    def learn(self):
        self.b_users=numpy.zeros(self.user_n)
        self.b_items=numpy.zeros(self.item_n)
        self.b=numpy.mean(self.R[numpy.where(self.R!=0)])
        
        self.P=numpy.random.normal(scale=1./self.K,size=(self.user_n,self.K))
        self.Q=numpy.random.normal(scale=1./self.K,size=(self.item_n,self.K))
        
        self.samples=[]
        for i in range (self.user_n):
            for j in range (self.item_n):
                if (self.R[i][j])!=0:
                    self.samples.append((i,j,self.R[i][j]))

        for i in range (self.iterations):
            numpy.random.shuffle(self.samples)
            self.sgd()
        self.leastCertain()
            
    def leastCertain(self):
        Q=numpy.absolute(self.complete())
        print("Least Certain Value Indexes: ")
        print(numpy.where(Q==Q.min())[0], numpy.where(Q==Q.min())[1])
        
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.b_users[i]+self.b_items[j]+self.b+self.P[i,:].dot(self.Q[j,:].T)
            e = (r - prediction)
            
            self.b_users[i]+=self.alpha*(e-self.beta*self.b_users[i])
            self.b_items[j]+=self.alpha*(e-self.beta*self.b_items[j])
            
            self.P[i,:]+=self.alpha*(e*self.Q[j,:]-self.beta*self.P[i,:])
            self.Q[j,:]+=self.alpha*(e*self.P[i,:]-self.beta*self.Q[j,:])
            
    def get_rating(self,i,j):
        return self.b_users[i]+self.b_items[j]+self.b+self.P[i,:].dot(self.Q[j,:].T)
    
    def complete(self):
        return self.P.dot(self.Q.T)+self.b+self.b_users[:,numpy.newaxis]+self.b_items[numpy.newaxis,:]
        

In [46]:
M = numpy.array([
    [1, -1, 0, 1],
    [1, 0, -1, 1],
    [-1, -1, 1, 1],
    [1, 0, 0, -1],
    [0, 1, 0, -1],
])

mf = matrix_factorization(M, 0.1, 0.01, 20, 2)

mf.learn()

print(numpy.around(mf.complete(),2))


Least Certain Value Indexes: 
[3] [2]
[[ 0.89 -0.88 -0.61  1.07]
 [ 1.04 -1.   -0.98  0.97]
 [-0.93 -1.06  1.01  0.97]
 [ 1.    0.81  0.04 -0.99]
 [ 1.42  0.96 -0.21 -1.  ]]
